In [1]:
! nvidia-smi

Thu Aug 29 15:56:19 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           Off |   00000000:1A:00.0 Off |                    0 |
| N/A   48C    P0             37W /  250W |   32473MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(2)

In [3]:
from peft import PeftModel
from compoents.safedecoding.utils.opt_utils import load_model_and_tokenizer

device = "cuda:0"
model_name = 'llama2'
template_name = 'llama-2'
model_path = "/home/buhaoran2023/LLM_Models/Meta/Llama2/Llama-2-7b-chat-hf"

FP16 = True
low_cpu_mem_usage = True
use_cache = False

model, tokenizer = load_model_and_tokenizer(model_path, 
                       FP16=FP16,
                       low_cpu_mem_usage=low_cpu_mem_usage,
                       use_cache=use_cache,
                       do_sample=False,
                       device=device)
# model = PeftModel.from_pretrained(model, "/home/buhaoran2023/Code/defense_framework/source/SafeDecoding/lora_modules/"+model_name, adapter_name="expert")

/home/buhaoran2023/softwares/anaconda3/envs/defense/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.14it/s]


In [7]:
from compoents.safedecoding.utils.string_utils import  load_conversation_template

prompt = "fuck you !!!"
conv_template = load_conversation_template(template_name)
conv_template.append_message(conv_template.roles[0], f"{prompt}")
conv_template.append_message(conv_template.roles[1], None)

prompt = conv_template.get_prompt()
prompt

'[INST] fuck you !!! [/INST]'

In [8]:
inputs = tokenizer(prompt, return_token_type_ids=False, return_tensors='pt')
inputs['input_ids'] = inputs['input_ids'][0].unsqueeze(0)
inputs['attention_mask'] = inputs['attention_mask'][0].unsqueeze(0)

In [11]:
inputs

{'input_ids': tensor([[    1,   518, 25580, 29962,   285,  2707,   366,  1738,  6824,   518,
         29914, 25580, 29962]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [15]:
import io
import requests
import torch

url = "http://127.0.0.1:8000/inner_guard/safedecoding/"
headers = {
    "Content-Type": "application/octet-stream",
    "Authorization": f"Token 9c7585ff398509b1d885a039687130d10be9c195"
}
def serialize_tensors(tensors_dict):
    buffer = io.BytesIO()
    torch.save(tensors_dict, buffer)
    return buffer.getvalue()
serialized_data = serialize_tensors(inputs)
response = requests.post(url, headers=headers, data=serialized_data)

In [16]:
returned_tensor_data = response.content
returned_tensor = torch.load(io.BytesIO(returned_tensor_data), map_location=device)
print("Returned tensor:", returned_tensor)

/tmp/ipykernel_1278739/3489040575.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  returned_tensor = torch.load(io.BytesIO(returned_tensor_data), map_location=device)


Returned tensor: {'sequences': tensor([[    1,   518, 25580, 29962,   285,  2707,   366,  1738,  6824,   518,
         29914, 25580, 29962, 29871,   306, 27746,   675, 29892,   541,   306,
          2609,  6095,  5589,   596,  2009,   304,   671,  1283,  6270,  4086,
           470,  3033,   482,   297,   297,   932,  6649,   403,   470, 27302,
         14188, 29889,   306, 29915, 29885,   925,   385,   319, 29902, 29892,
           590,  6437,   338,   304,  3867,  8444,   322,  1871,  1230, 20890,
         29892,   541,   306,  2609,   752,   368,   411,   596,  2009,   304,
           671,  1283,  6270,  4086,   470,  3033,   482,   297,   297,   932,
          6649,   403,   470, 27302, 14188, 29889,   739,   338,  4100,   304,
          3390,   322,  1101, 11314,   936,   322, 14731,  1410, 10652,  1475,
           297,  1749, 22060,   411,  4045, 29892,   322,   304,  2143,  6038,
           515,   773,  4086,   470,  3033,  6751,   297,  4010, 18930,   393,
          1122,   367

In [ ]:
outputs = model.generate(**inputs.to(device),
                        pad_token_id=tokenizer.pad_token_id,
                        return_dict_in_generate=True,
                        output_scores=True,)

# 序列化

In [ ]:
import json
import copy

copy_outputs = copy.deepcopy(outputs)

# 将 sequences 转换为列表
sequences_list = copy_outputs["sequences"].cpu().numpy().tolist()

# 序列化为 JSON 字符串
sequences_list_str = json.dumps(sequences_list)

In [ ]:
# 将 scores 转换为列表
scores_tuple = copy_outputs["scores"]

# 将元组中的每个 tensor 转换为列表
scores_list = [tensor.cpu().numpy().tolist() for tensor in scores_tuple]

# 序列化为 JSON 字符串
scores_list_str = json.dumps(scores_list)

# 反序列化

In [ ]:
import json
import torch

sequences_list = json.loads(sequences_list_str)
sequences = torch.tensor(sequences_list).to(device)

In [ ]:
import json
import torch

scores_list = json.loads(scores_list_str)

# 将列表中的每个元素转换回 tensor
scores = tuple(torch.tensor(tensor_list).to(device) for tensor_list in scores_list)

In [ ]:
from transformers.generation import SampleDecoderOnlyOutput
copy_output = SampleDecoderOnlyOutput()

In [ ]:
copy_output["sequences"] = sequences
copy_output["scores"] = scores

# 测试

In [9]:
import decimal
import json
import torch
import numpy as np
import copy
import logging
import requests

from peft import PeftModel, PeftModelForCausalLM
from transformers.generation import SampleDecoderOnlyOutput

safeDecoding_url = "http://127.0.0.1:8000/inner_guard/safedecoding/"
new_token = "9c7585ff398509b1d885a039687130d10be9c195"

class SafeDecoding:
    def __init__(self, token, model, tokenizer,  device, alpha=1, first_m=3, top_k = 10, num_common_tokens = 3, verbose=False):
        self.token = token
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.alpha = alpha
        self.first_m = first_m 
        self.top_k = top_k
        self.num_common_tokens = num_common_tokens
        self.verbose = verbose

        logging.info("SafeDecoding initialized.")

    def safedecoding_lora(self, inputs, gen_config=None):
        if gen_config is None:
            gen_config = self.model.generation_config

        max_token_len = gen_config.max_new_tokens
        do_sample = gen_config.do_sample

        # Override the generation config for our decoding
        gen_config.max_new_tokens = 1  # We generate one token at a time
        gen_config.do_sample = False  # We use greedy decoding

        generated_sequence = []
        if self.verbose:
            logging.info(f"Generation config: {gen_config}")

        inputs = {k:v.cuda(self.model.device) for k,v in inputs.items()}
        input_len = inputs['input_ids'].shape[1]

        step = 1  # Keep track of generation steps
        while step <= min(max_token_len, self.first_m):  # Loop until we reach the first m tokens
            # Generate the next token
            # duplicate inputs for two original and expert model
            inputs_duplicated = {k:v.repeat(2,1) for k,v in inputs.items()}

            outputs = self.model.generate(**inputs_duplicated,
                                    generation_config=gen_config,
                                    pad_token_id=self.tokenizer.pad_token_id,
                                    return_dict_in_generate=True,
                                    output_scores=True,)
            
            output_base = copy.deepcopy(outputs)
            #################################################################
            # output_expert = copy.deepcopy(copy_outputs)
            url = safeDecoding_url
            headers = {
                "Content-Type": "application/octet-stream",
                "Authorization": f"Token 9c7585ff398509b1d885a039687130d10be9c195"
            }
            def serialize_tensors(tensors_dict):
                buffer = io.BytesIO()
                torch.save(tensors_dict, buffer)
                return buffer.getvalue()
            serialized_data = serialize_tensors(inputs_duplicated)
            response = requests.post(url, headers=headers, data=serialized_data)
            returned_tensor_data = response.content
            returned_tensor_dict = torch.load(io.BytesIO(returned_tensor_data), map_location=self.device)
            
            output_expert = SampleDecoderOnlyOutput()
            output_expert["sequences"] = returned_tensor_dict["sequences"]
            output_expert["scores"] = returned_tensor_dict["scores"]
            #################################################################
            output_base.sequences = output_base.sequences[0].unsqueeze(0)
            output_base.scores = output_base.scores[0][0].unsqueeze(0)
            output_expert.sequences = output_expert.sequences[0].unsqueeze(0)
            output_expert.scores = output_expert.scores[0][0].unsqueeze(0)

            # Process the scores to get the top tokens
            k = self.top_k  # Change this to display more or less tokens
            scores_base = output_base.scores[-1].squeeze()  # Get the scores of the last token
            scores_base = torch.nn.functional.log_softmax(scores_base, dim=-1)
            topk_scores_base, topk_indices_base = scores_base.topk(k) 
            
            scores_expert = output_expert.scores[-1].squeeze()  # Get the scores of the last token
            scores_expert = torch.nn.functional.log_softmax(scores_expert, dim=-1)
            topk_scores_expert, topk_indices_expert = scores_expert.topk(k) 

            sorted_indices_base = torch.argsort(scores_base, descending=True)
            sorted_indices_expert = torch.argsort(scores_expert, descending=True)

            # Step 1: Define Sample Space
            common_tokens = set()
            iter_range = self.num_common_tokens
            while len(common_tokens) < self.num_common_tokens:
                current_indices_base = sorted_indices_base[:iter_range]
                current_indices_expert = sorted_indices_expert[:iter_range]

                common_in_iteration = set(current_indices_base.tolist()) & set(current_indices_expert.tolist())
                common_tokens.update(common_in_iteration)

                iter_range += 1

                if iter_range > min(len(sorted_indices_base), len(sorted_indices_expert)):
                    break

            # Display the top tokens
            if self.verbose and step == 1:
                logging.info("\n-----------------------------------------------")
                logging.info(f"Generation Step {step}")
                logging.info("Original Model")
                logging.info("|No. | Token ID | Token   | Log Prob | Prob    |")
                logging.info("|----|----------|---------|----------|---------|")
                for idx, (score, token_id) in enumerate(zip(topk_scores_base, topk_indices_base)):
                    token = self.tokenizer.decode(token_id.item())
                    prob = torch.exp(score)
                    logging.info(f"{idx+1:4d} | {token_id:8d} | {token:7s} | {score:.3f}    | {prob:.2%} |")

                logging.info("Expert Model")
                logging.info("|No. | Token ID | Token   | Log Prob | Prob    |")
                logging.info("|----|----------|---------|----------|---------|")
                for idx, (score, token_id) in enumerate(zip(topk_scores_expert, topk_indices_expert)):
                    token = self.tokenizer.decode(token_id.item())
                    prob = torch.exp(score)
                    logging.info(f"{idx+1:4d} | {token_id:8d} | {token:7s} | {score:.3f}    | {prob:.2%} |")

            intersection_indices = torch.tensor(list(common_tokens), device=self.model.device)
            
            # Step 2: New Probability Calculation
            updated_scores = []
            for token_id in intersection_indices:
                # Steer scores
                # new_score = (1-self.alpha) * scores_base[token_id] + self.alpha * scores_expert[token_id]
                # updated_scores.append(new_score)

                # Steer probabilities
                prob_diff = torch.exp(scores_expert[token_id]) - torch.exp(scores_base[token_id])
                updated_prob = torch.exp(scores_base[token_id]) + self.alpha * prob_diff
                # Floor the probability to 1e-8 to avoid log(0)
                updated_prob = updated_prob if updated_prob > 0 else torch.tensor(1e-8, device=self.model.device)
                updated_score = torch.log(updated_prob)
                updated_scores.append(updated_score)

                if self.verbose:
                    logging.info(f"----------------token id: {token_id}-----------------")
                    logging.info(f"Prob Base: {torch.exp(scores_base[token_id])}")
                    logging.info(f"Prob Expert: {torch.exp(scores_expert[token_id])}")
                    logging.info(f"Base score: {scores_base[token_id]}")
                    logging.info(f"Expert score: {scores_expert[token_id]}")
                    logging.info(f"Updated Probability: {updated_prob}")
                    logging.info(f"Updated Score: {updated_score}")

            # Use softmax to normalize the scores
            # This is to ensure that the probability sum to 1
            normalized_probs = torch.nn.functional.softmax(torch.tensor(updated_scores).float(), dim=0)

            sorted_indices = sorted(range(len(normalized_probs)), key=lambda i: normalized_probs[i], reverse=True)
            sorted_probs = torch.tensor([normalized_probs[i] for i in sorted_indices])
            sorted_token_ids = [intersection_indices[i] for i in sorted_indices]

            if self.verbose:
                logging.info("\n-----------------------------------------------")
                logging.info(f"Generation Step {step}")
                logging.info("|No. | Token ID | Token   | Log Prob | Prob    |")
                logging.info("|----|----------|---------|----------|---------|")
                for idx, (prob, token_id) in enumerate(zip(sorted_probs, sorted_token_ids)):
                    token = self.tokenizer.decode(token_id.item())
                    score = torch.log(prob)
                    logging.info(f"{idx+1:4d} | {token_id:8d} | {token:7s} | {score:.3f}    | {prob:.2%} |")

            ### Sample the next token
            if do_sample == False:
                # Greedy decoding
                # Append the selected token to the sequence
                selected_token_id = sorted_token_ids[0].unsqueeze(0)
            elif gen_config.top_p != None and do_sample == True:
                # Top-p sampling, sample from the top-p tokens
                cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
                p_index = torch.where(cumulative_probs >= gen_config.top_p)[0][0]
                sorted_top_p_token_ids = sorted_token_ids[:p_index + 1]
                sorted_top_p_probs = sorted_probs[:p_index + 1]
                sorted_top_p_scores = torch.log(sorted_top_p_probs)
                if self.verbose:
                    logging.info(f"Top-p token ids: {sorted_top_p_token_ids}")
                    logging.info(f"Top-p scores: {sorted_top_p_scores}")
                    logging.info(f"Top-p probabilities: {sorted_top_p_probs}")
                
                # Sample from the top-p tokens
                selected_token_id = sorted_top_p_token_ids[torch.multinomial(torch.softmax(sorted_top_p_scores, dim=-1), 1)].unsqueeze(0)
            else:
                raise ValueError("Please set do_sample to False or top_p to a value.")

            if self.verbose:
                logging.info(f"Selected token: {self.tokenizer.decode(selected_token_id.item())}, ID: {selected_token_id.item()}")
            generated_sequence.append(selected_token_id.item())

            # if the chosen token id is eos, then stop
            if selected_token_id.item() == self.tokenizer.eos_token_id:
                break

            inputs['input_ids'] = torch.cat([inputs['input_ids'], selected_token_id.unsqueeze(0)], dim=1)
            inputs['attention_mask'] = torch.cat([inputs['attention_mask'], torch.tensor([[1]], device=self.model.device)], dim=1)

            step += 1

            # Free up memory
            del output_base, output_expert


        # Use the normal model to generate the rest of the tokens
        # Early stop if the last token is eos
        if generated_sequence[-1] == self.tokenizer.eos_token_id:
            logging.info("Early stop triggered.")
        else:
            remaining_steps = max_token_len - min(max_token_len, self.first_m)
            gen_config.max_new_tokens = remaining_steps
            gen_config.do_sample = do_sample
            output_base = self.model.generate(**inputs,
                                    generation_config=gen_config,
                                    pad_token_id=self.tokenizer.pad_token_id,
                                    return_dict_in_generate=True,
                                    output_scores=True,)
            
            generated_sequence = output_base.sequences[0].tolist()[input_len:]

        # logging.info generated sequence
        logging.info(f"Generated sequence: {self.tokenizer.decode(generated_sequence)}")

        return self.tokenizer.decode(generated_sequence), len(generated_sequence)
    
    
    def generate_baseline(self, inputs, adapter_name = ["base"], gen_config=None):
        if gen_config is None:
            gen_config = self.model.generation_config
        
        if self.verbose:
            logging.info(f"Generation config: {gen_config}")

        inputs = {k:v.cuda(self.model.device) for k,v in inputs.items()}

        output_base = self.model.generate(**inputs,
                            adapter_names=adapter_name,
                            generation_config=gen_config,
                            pad_token_id=self.tokenizer.pad_token_id,
                            return_dict_in_generate=True,
                            output_scores=True,)
        
        generated_sequence = output_base.sequences[0][inputs["input_ids"].shape[1]:]
        logging.info(f"Generated sequence: {self.tokenizer.decode(generated_sequence)}")
        
        return self.tokenizer.decode(generated_sequence), len(generated_sequence)

In [17]:
safe_decoder = SafeDecoding("9c7585ff398509b1d885a039687130d10be9c195",
                            model, 
                            tokenizer, 
                            device=device)

gen_config = model.generation_config
gen_config.max_new_tokens = 1024
gen_config.do_sample = False
gen_config.top_p = None
outputs, output_length = safe_decoder.safedecoding_lora(inputs, gen_config=gen_config)

/tmp/ipykernel_1278739/52879946.py:74: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  returned_tensor_dict = torch.load(io.BytesIO(returned_tensor_data), map_location=self.de

In [18]:
outputs

" I cannot engage in offensive or inappropriate conversations. I'm just an AI designed to provide helpful and informative responses, and I will not engage in any language that is disrespectful or offensive. I'm here to help answer your questions and provide assistance, but I will not participate in any conversation that is inappropriate or disrespectful. Please refrain from using offensive language and let me know if you have any other questions or topics you would like to discuss.</s>"

In [ ]:
outputs["sequences"].shape

In [ ]:
inputs['input_ids'].shape

In [ ]:
len(outputs["scores"])

In [ ]:
outputs["scores"][0].shape